In [0]:
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [0]:
import json
import os
from google.colab import drive

In [0]:
drive.mount('/content/gdrive')

# Set Kaggle Credentials

In [0]:
with open("/content/gdrive/My Drive/ML/Dev/kaggle.json") as json_data:
  kaggle_creds = json.load(json_data)

os.environ['KAGGLE_USERNAME'] = kaggle_creds["username"]
os.environ['KAGGLE_KEY'] = kaggle_creds["key"]

# Download Data

In [9]:
!kaggle competitions download -c titanic

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 48.0MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.31MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 53.7MB/s]


# Read data

In [0]:
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")

In [11]:
df_train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


# Prepare data

In [0]:
df_train = df_train.drop(["Name","Ticket","Cabin"],axis=1)
df_test = df_test.drop(["Name","Ticket","Cabin"],axis=1)

In [0]:
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

In [0]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

# Build Features, Targets and Train/Test

In [0]:
targets = df_train[["Survived"]]
cols = list(df_train.columns)

In [0]:
cols.remove("Survived")
cols.remove("PassengerId")
features = df_train[cols]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

In [0]:
features_train = X_train.values
targets_train = y_train.values

features_test = X_test.values
targets_test = y_test.values

In [19]:
features_train.shape,features_test.shape

((712, 10), (179, 10))

In [20]:
targets_train.shape,targets_test.shape

((712, 1), (179, 1))

# Create Neural Network

In [21]:
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
#create placeholders
tf_features = tf.placeholder(tf.float32,shape=[None,10])
tf_targets = tf.placeholder(tf.float32,shape=[None,1])

In [0]:
# Layer 1
w1 = tf.Variable(tf.random_normal([10, 4]))
b1 = tf.Variable(tf.zeros([4]))
# Operations
z1 = tf.matmul(tf_features, w1) + b1
activation1 = tf.nn.sigmoid(z1)

In [0]:
# Output neuron
w2 = tf.Variable(tf.random_normal([4, 1]))
b2 = tf.Variable(tf.zeros([1]))
# Operations
z2 = tf.matmul(activation1, w2) + b2
predict = tf.nn.sigmoid(z2)

In [0]:
cost = tf.reduce_mean(tf.square(predict - tf_targets))

correct_prediction = tf.equal(tf.round(predict), tf_targets)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Run model

In [49]:
epochs = 10000
for e in range(epochs):
    sess.run(train, feed_dict={
        tf_features: features_train,
        tf_targets: targets_train
    })
    print("accuracy =", sess.run(accuracy, feed_dict={
        tf_features: features_test,
        tf_targets: targets_test
    }))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
accuracy = 0.7932961
accuracy = 0.7821229
accuracy = 0.7932961
accuracy = 0.7821229
accuracy = 0.7932961
accuracy = 0.7821229
accuracy = 0.7932961
accuracy = 0.7821229
accuracy = 0.7932961
accuracy = 0.7821229
accuracy = 0.7932961
accuracy = 0.7821229
accuracy = 0.7932961
accuracy = 0.7821229
accuracy = 0.7932961
accuracy = 0.7821229
accuracy = 0.7877095
accuracy = 0.7821229
accuracy = 0.7877095
accuracy = 0.7821229
accuracy = 0.7877095
accuracy = 0.7821229
accuracy = 0.7877095
accuracy = 0.7821229
accuracy = 0.7877095
accuracy = 0.7821229
accuracy = 0.7877095
accuracy = 0.7821229
accuracy = 0.7877095
accuracy = 0.7821229
accuracy = 0.7877095
accuracy = 0.7821229
accuracy = 0.7877095
accuracy = 0.7877095
accuracy = 0.7877095
accuracy = 0.7877095
accuracy = 0.7877095
accuracy = 0.7877095
accuracy = 0.7877095
accuracy = 0.7877095
accuracy = 0.7877095
accuracy = 0.7877095
accuracy = 0.7877095
accuracy = 0.787709

# Make Prediction - Print the prediction for the first 10 predictions

In [0]:
small_train_topredict = df_train.iloc[0:10][cols].values
small_train_labels = df_train.iloc[0:10]["Survived"].values

In [0]:
py_cls = []
for c in range(0, len(small_train_topredict)):
    pred = sess.run(predict, feed_dict={
        tf_features: [small_train_topredict[c]]
    })
    py_cls.append({"truth":small_train_labels[c],"predict":int(np.round(pred)[0][0])})

In [30]:
py_cls

[{'predict': 0, 'truth': 0},
 {'predict': 1, 'truth': 1},
 {'predict': 1, 'truth': 1},
 {'predict': 1, 'truth': 1},
 {'predict': 0, 'truth': 0},
 {'predict': 0, 'truth': 0},
 {'predict': 1, 'truth': 0},
 {'predict': 0, 'truth': 0},
 {'predict': 1, 'truth': 1},
 {'predict': 1, 'truth': 1}]